In [4]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

In [5]:
@model function smoothing(n, x0, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    c = constvar(1.0)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [6]:
P = 1.0

n = 10_000
k = 500
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [7]:
function inference(; data, k, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, P, options = (limit_stack_depth = k, ));

    buffer    = Vector{Marginal}(undef, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, (ms) -> copyto!(buffer, ms))
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return buffer
end

inference (generic function with 1 method)

In [8]:
@benchmark res = inference(
    data = $data,
    k = $k,
    x0 = NormalMeanVariance(0.0, 10000.0),
    P = $P
)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  708.529 ms … 807.643 ms  ┊ GC (min … max): 34.56% … 43.42%
 Time  (median):     745.513 ms               ┊ GC (median):    39.67%
 Time  (mean ± σ):   757.621 ms ±  33.244 ms  ┊ GC (mean ± σ):  39.66% ±  3.69%

  █                  █ ██               █         █           █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█ ▁
  709 ms           Histogram: frequency by time          808 ms <

 Memory estimate: 287.14 MiB, allocs estimate: 4501258.

In [9]:
@benchmark res = inference(
    data = $data,
    k = $k,
    x0 = NormalMeanVariance(0.0, 10000.0),
    P = $P
)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  613.312 ms … 815.884 ms  ┊ GC (min … max): 24.49% … 44.75%
 Time  (median):     762.562 ms               ┊ GC (median):    43.44%
 Time  (mean ± σ):   740.469 ms ±  84.183 ms  ┊ GC (mean ± σ):  38.40% ±  7.40%

  ▁       ▁                        ▁          ▁              ▁█  
  █▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁██ ▁
  613 ms           Histogram: frequency by time          816 ms <

 Memory estimate: 287.14 MiB, allocs estimate: 4501258.